In [ ]:
library('palmerpenguins')
library('tidymodels')
library('tidyverse')

theme_set(theme_grey(base_size = 14))

## One categorical and one continuous explanatory predictor

In [ ]:
penguins2 = penguins |>
    filter(species %in% c('Adelie', 'Gentoo')) |>
    select(bill_depth_mm, species, bill_length_mm) |>
    mutate(species = forcats::fct_drop(species)) |>
    drop_na()

penguins2 |> head()

In [ ]:
rec = recipe(bill_length_mm ~ species + bill_depth_mm, data = penguins2) |>
    step_dummy(species)

rec

In [ ]:
rec_prepped  = prep(rec, penguins2) 

rec_prepped |> 
    juice() |>
    head()

In [ ]:
mod = linear_reg() |> set_engine('lm')

In [ ]:
mod_fit = mod |> fit(bill_length_mm ~ ., data = juice(rec_prepped))

tidy(mod_fit)

In [ ]:
penguins2_with_pred = augment(mod_fit, juice(rec_prepped))

penguins2_with_pred |> head()

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 6)

ggplot(penguins2_with_pred) +
    geom_point(aes(x = bill_depth_mm, y = bill_length_mm, color = factor(species_Gentoo))) +
    labs(color = 'is Gentoo?') +
    geom_line(aes(x = bill_depth_mm, y = .pred, group = species_Gentoo), 
              color = 'steelblue', linewidth = 1.2) +
    geom_segment(aes(x = bill_depth_mm, xend = bill_depth_mm, y = .pred, yend = bill_length_mm), 
                 color = 'firebrick',
                 alpha = 0.25)
     

## Adding an interaction term

In [ ]:
rec = recipe(bill_length_mm ~ species + bill_depth_mm, data = penguins2) |>
    step_dummy(species) |>
    step_interact(terms = ~ bill_depth_mm:species_Gentoo)

rec

In [ ]:
rec_prepped  = prep(rec, penguins2) 

rec_prepped |> 
    juice() |>
    tail()

In [ ]:
mod = linear_reg() |> set_engine('lm')

In [ ]:
mod_fit = mod |> fit(bill_length_mm ~ ., juice(rec_prepped))

mod_fit

In [ ]:
penguins2_with_pred = augment(mod_fit, juice(rec_prepped))

penguins2_with_pred |> head()

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 6)

ggplot(penguins2_with_pred) +
    geom_point(aes(x = bill_depth_mm, y = bill_length_mm, color = factor(species_Gentoo))) +
    labs(color = 'is Gentoo?') +
    geom_line(aes(x = bill_depth_mm, y = .pred, group = species_Gentoo), 
              color = 'steelblue', linewidth = 1.2) +
    geom_segment(aes(x = bill_depth_mm, xend = bill_depth_mm, y = .pred, yend = bill_length_mm), 
                 color = 'firebrick',
                 alpha = 0.25)
     